<a href="https://colab.research.google.com/github/sulthonpriyan/CapstoneProjectTeamC23-PS081/blob/main/Medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Get Started with TensorFlow Recommenders and Matrix Factorization**
A hands-on tutorial on recommender systems with TensorFlow

In this post, we will work with the TensorFlow tutorial where we will try to go deeper by showing:
*   How to start with a pandas data frame instead of a TensorFlow datasets
*   How to get the Users’ and Items’ Embeddings
*   How to find the expected score for every item for each user
*   How to make recommendations for each user
*   How to find similar items
*   How to save and load the TensorFlow model

# **Preprocess the Data**
We will work with the [MovieLens dataset](https://https://grouplens.org/datasets/movielens/100k/), collected by the GroupLens Research Project at the University of Minnesota. Our goal is to build a model that suggests movies to users. We will keep the user-item pairs where the rating is above 3 and this is because we would like to recommend movies that the user is likely to watch but also like.

In [5]:
import pandas as pd
 
# load the rating data
from google.colab import drive
drive.mount('/content/drive')
columns = ['user_id', 'item_id', 'rating', 'timestamp']
ratings = pd.read_csv('/content/drive/MyDrive/ml-100k/u.data', sep='\t', names=columns)
ratings.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
# load the movies data
 
columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
 
movies = pd.read_csv('/content/drive/MyDrive/ml-100k/u.item', sep='|', names=columns, encoding='latin-1')
movies = movies[['item_id', 'movie title']]
movies.head()

,item_id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [7]:
# join the ratings with the movies
ratings = pd.merge(ratings, movies, on='item_id')
 
 
# keep only moviews with a rating greater than 3
ratings = ratings[ratings.rating>3]
 
 
# keep only the user id and the movie title columns
ratings = ratings[['movie title', 'user_id']].reset_index(drop=True)
 
ratings

,movie title,user_id
0,Kolya (1996),226
1,Kolya (1996),306
2,Kolya (1996),296
3,Kolya (1996),34
4,Kolya (1996),271
...,...,...
55370,Brothers in Trouble (1995),655
55371,Everest (1998),532
55372,Everest (1998),416
55373,"Butcher Boy, The (1998)",655


In [8]:
# save to a csv file
 
ratings.to_csv('ratings.csv', index=False)
movies.to_csv('movies.csv', index=False)

# **Build the Model**
The idea is to build a retrieval model using user and item embeddings. We will work with the TensorFlow-recommenders library.

In [9]:
!pip install -q tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.2 MB/s eta 0:00:00


In [10]:
from typing import Dict, Text
 
import numpy as np
import pandas as pd
import tensorflow as tf
 
import tensorflow_recommenders as tfrs 

Since we installed the tensorflow-recommenders on Colab, we will load the `ratings.csv` and `movies.csv` that we generated in the previous step.

In [11]:
# read the csv files as pandas data frames
ratings_df = pd.read_csv('ratings.csv')
movies_df = pd.read_csv('movies.csv')
 
 
ratings_df.rename(columns = {'movie title': 'movie_title'}, inplace=True)
movies_df.rename(columns = {'movie title': 'movie_title'},  inplace=True)

Now, we will convert the pandas data frames to TensorFlow datasets.

In [12]:
# convert them to tf datasets
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_df))
movies = tf.data.Dataset.from_tensor_slices(dict(movies_df))

Let’s have a look at our data:

In [13]:
# get the first rows of the movies dataset
for m in movies.take(5):
  print(m)

{'item_id': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Toy Story (1995)'>}
{'item_id': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'GoldenEye (1995)'>}
{'item_id': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Four Rooms (1995)'>}
{'item_id': <tf.Tensor: shape=(), dtype=int64, numpy=4>, 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Get Shorty (1995)'>}
{'item_id': <tf.Tensor: shape=(), dtype=int64, numpy=5>, 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Copycat (1995)'>}


In [14]:
# get the first rows of the ratings dataset
for r in ratings.take(5):
  print(r)

{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Kolya (1996)'>, 'user_id': <tf.Tensor: shape=(), dtype=int64, numpy=226>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Kolya (1996)'>, 'user_id': <tf.Tensor: shape=(), dtype=int64, numpy=306>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Kolya (1996)'>, 'user_id': <tf.Tensor: shape=(), dtype=int64, numpy=296>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Kolya (1996)'>, 'user_id': <tf.Tensor: shape=(), dtype=int64, numpy=34>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Kolya (1996)'>, 'user_id': <tf.Tensor: shape=(), dtype=int64, numpy=271>}


Let’s keep the basic features of our model.

In [15]:
# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

# **Build vocabularies to convert user ids and movie titles into integer indices for embedding layers**
For our model, we need to assign indices for the unique users and movies. Note that we add an extra index for the unknown users and movies respectively.

In [16]:
user_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))
 
 
movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

# **Create the Model**
We will work with the tfrs.Model by implementing the compute_loss method.

In [17]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.
 
  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()
 
    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model
 
    # Set up a retrieval task.
    self.task = task
 
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.
 
    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])
 
    return self.task(user_embeddings, movie_embeddings)

We have to define the user_model and moview_model which are sequential models for generating the embeddings. Finally, the objective of the task is a retrieval model.

In [18]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])
 
# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

# **Fit the Model**
The last step is to build the model, train it and make some predictions.

In [19]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))
 
# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

Epoch 1/3
14/14 [==============================] - 24s 2s/step - factorized_top_k/top_1_categorical_accuracy: 3.6117e-05 - factorized_top_k/top_5_categorical_accuracy: 7.4041e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0014 - factorized_top_k/top_50_categorical_accuracy: 0.0089 - factorized_top_k/top_100_categorical_accuracy: 0.0198 - loss: 32272.2602 - regularization_loss: 0.0000e+00 - total_loss: 32272.2602
Epoch 2/3
14/14 [==============================] - 19s 1s/step - factorized_top_k/top_1_categorical_accuracy: 3.7923e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0043 - factorized_top_k/top_10_categorical_accuracy: 0.0103 - factorized_top_k/top_50_categorical_accuracy: 0.0647 - factorized_top_k/top_100_categorical_accuracy: 0.1313 - loss: 33919.5090 - regularization_loss: 0.0000e+00 - total_loss: 33919.5090
Epoch 3/3
14/14 [==============================] - 20s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0048 - factorized_top_k/top_5_categorical_accu

# **Make Predictions**
Let’s make predictions for the `user_id=42`.

In [20]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))
 
# Get some recommendations.
_, titles = index(np.array([42]))
print(f"Top 10 recommendations for user 42: {titles[0, :10]}")

Top 10 recommendations for user 42: [b'Forget Paris (1995)' b'Only You (1994)' b'Milk Money (1994)'
 b'Net, The (1995)' b'Unforgettable (1996)' b'Murder in the First (1995)'
 b'Associate, The (1996)' b'Clean Slate (1994)'
 b'Far From Home: The Adventures of Yellow Dog (1995)'
 b'Mrs. Doubtfire (1993)']


# **Get Users’ and Movies’ Embeddings**
It is very useful to get and work with the users’ and movies’ embeddings since they can be used in several tasks. The embeddings are simply the weights of the model. Keep in mind that each embedding corresponds to a vocabulary value.

**Users’ Embeddings**

In [21]:
# get the users embeddings
users_embdeddings = user_model.weights[1].numpy()
 
# get the mapping of the user ids from the vocabulary
users_idx_name = user_ids_vocabulary.get_vocabulary()
 
# print the shape
users_embdeddings.shape

(943, 64)

**Movies’ Embeddings**

In [22]:
# get the movies embeddings
movies_embdeddings = movie_model.weights[1].numpy()
 
# get the mapping of the movie tiles from the vocabulary
movie_idx_name = movie_titles_vocabulary.get_vocabulary()
 
# print the shape of the movies embeddings
movies_embdeddings.shape

(1665, 64)

# **Find Similar Movies Based on Movies Embeddings**
At this step, we will return the pairs of the most similar movies. Of course, you can get the most similar movies for any specific movie.

In [23]:
from sklearn.metrics import pairwise_distances
 
# get the cosine similarity of all pairs
movies_similarity = 1-pairwise_distances(movies_embdeddings, metric='cosine')
 
# get the upper triangle in order to take the unique pairs
movies_similarity = np.triu(movies_similarity)

Get the pairs of the most similar movies with a threshold of cosine similarity **greater than 0.8.**

In [24]:
Movie_A = np.take(movie_idx_name, np.where((movies_similarity>0.8))[0])
Movie_B = np.take(movie_idx_name, np.where((movies_similarity>0.8))[1])

similar_movies = pd.DataFrame({'Movie_A':Movie_A, 'Movie_B':Movie_B})
similar_movies.head(100)

,Movie_A,Movie_B
0,[UNK],[UNK]
1,Ulee's Gold (1997),Ulee's Gold (1997)
2,That Darn Cat! (1997),That Darn Cat! (1997)
3,"Substance of Fire, The (1996)","Substance of Fire, The (1996)"
4,Sliding Doors (1998),Sliding Doors (1998)
...,...,...
95,"Wedding Singer, The (1998)","Wedding Singer, The (1998)"
96,"Wedding Gift, The (1994)","Wedding Gift, The (1994)"
97,Wedding Bell Blues (1996),Wedding Bell Blues (1996)
98,Waterworld (1995),Waterworld (1995)


# **Get User’s Recommendations with Matrix Multiplication**
Since we have built the user matrix and the movie matrix, we can multiply the two tables in order to get a score for each user for every movie.

In [25]:
# get the product of users and movies embeddings
product_matrix = np.matmul(users_embdeddings, np.transpose(movies_embdeddings))
 
# get the shape of the product matrix 
product_matrix.shape

(943, 1665)

In essence, the above matrix is the estimated user-item matrix. Let’s get the top 10 recommended movies for the user=42 using the matrix estimated user-item matrix.

In [26]:
# score of movies for user 42
user_42_movies = product_matrix[users_idx_name.index(42),:]
 
# return the top 10 movies 
np.take(movie_idx_name, user_42_movies.argsort()[::-1])[0:10]

array(['Forget Paris (1995)', 'Only You (1994)', 'Milk Money (1994)',
       'Net, The (1995)', 'Unforgettable (1996)',
       'Murder in the First (1995)', 'Associate, The (1996)',
       'Clean Slate (1994)',
       'Far From Home: The Adventures of Yellow Dog (1995)',
       'Mrs. Doubtfire (1993)'], dtype='<U81')

As we can see, we got exactly the same results as above when we used the 
```
tfrs.layers.factorized_top_k.BruteForce.

```


If we want to **exclude the already watched movies,** we can work as follows.

In [27]:
seen_movies = ratings_df.query('user_id==42')['movie_title'].values
 
np.setdiff1d(np.take(movie_idx_name, user_42_movies.argsort()[::-1]), seen_movies, assume_unique=True)[0:10]


array(['Milk Money (1994)', 'Net, The (1995)',
       'Far From Home: The Adventures of Yellow Dog (1995)',
       'Mrs. Doubtfire (1993)', 'Mask, The (1994)',
       'Little Big League (1994)', 'I.Q. (1994)', 'Nine Months (1995)',
       'Nell (1994)', 'Crimson Tide (1995)'], dtype='<U81')

# **Save and Load the Model**
To deploy a model like this, we simply export the BruteForce layer we created above:

In [28]:
import tempfile
import os
# Export the query model.
with tempfile.TemporaryDirectory() as tmp:
  path = os.path.join(tmp, "model")
 
  # Save the index.
  tf.saved_model.save(index, path)
 
  # Load it back; can also be done in TensorFlow Serving.
  loaded = tf.saved_model.load(path)
 
  # Pass a user id in, get top predicted movie titles back.
  scores, titles = loaded([42])
 
  print(f"Recommendations: {titles[0][:10]}")

Recommendations: [b'Forget Paris (1995)' b'Only You (1994)' b'Milk Money (1994)'
 b'Net, The (1995)' b'Unforgettable (1996)' b'Murder in the First (1995)'
 b'Associate, The (1996)' b'Clean Slate (1994)'
 b'Far From Home: The Adventures of Yellow Dog (1995)'
 b'Mrs. Doubtfire (1993)']


As expected, we get the same recommendations for user 42.